# Modelos de Classificação - Hepatite

In [93]:
# Bibliotecas de manipualção e visualização de dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
from sklearn.preprocessing import StandardScaler
import seaborn as sns

# Classes dos modelo
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from mord import LogisticAT
from xgboost import XGBClassifier

# Essemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Funções de avaliação dos modelos
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score, plot_confusion_matrix, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from yellowbrick.classifier import ROCAUC

#SMOTE
from imblearn.over_sampling import SMOTE

#PerC
from PerC.src.data.build_dataset import build, normalize
from PerC.src.model.classification.perturbation import PerC_Mean, PerC_Covariance, PerC

# Seleção de Features e redução de dimencionalidade
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA


In [94]:
# dataset sem outliers
df_hepatite = pd.read_csv('HCV-Egy-Data-no-outlier.csv')

A remoção de outliers mostrou-se eficiente na performace do modelo aumentando em serca de 1% a acurácia.

In [95]:
df_hepatite.head()

,Unnamed: 0,Age,Gender,BMI,Fever,Nausea/Vomting,Headache,Diarrhea,FGba,Jaundice,...,ALT 24,ALT 36,ALT 48,ALT after 24 w,RNA Base,RNA 4,RNA 12,RNA EOT,RNA EF,Baselinehistological staging
0,1,46,1,29,1,2,2,1,2,2,...,113,57,123,44,40620,538635,637056,336804,31085,2
1,3,49,2,33,1,2,1,2,1,2,...,88,48,77,33,1041941,449939,585688,744463,582301,3
2,5,58,2,22,2,2,2,1,2,2,...,65,73,114,29,1157452,1086852,5,5,5,4
3,6,42,2,26,1,1,2,2,2,2,...,107,84,80,28,325694,1034008,275095,214566,635157,4
4,7,48,2,30,1,1,2,2,1,1,...,45,96,53,39,641129,72050,787295,370605,506296,3


Ao exportar o dataset sem outliers surgiu a coluna `Unnamed: 0` que será removida.

In [96]:
df_hepatite.drop('Unnamed: 0', axis=1, inplace=True)

In [97]:
df_hepatite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1381 entries, 0 to 1380
Data columns (total 28 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Age                           1381 non-null   int64  
 1   Gender                        1381 non-null   int64  
 2   BMI                           1381 non-null   int64  
 3   Fever                         1381 non-null   int64  
 4   Nausea/Vomting                1381 non-null   int64  
 5   Headache                      1381 non-null   int64  
 6   Diarrhea                      1381 non-null   int64  
 7   FGba                          1381 non-null   int64  
 8   Jaundice                      1381 non-null   int64  
 9   Ep                            1381 non-null   int64  
 10  WBC                           1381 non-null   int64  
 11  RBC                           1381 non-null   float64
 12  HGB                           1381 non-null   int64  
 13  Pla

### Seperação da váriável target do dataset

In [98]:
X = df_hepatite.drop('Baselinehistological staging', axis=1)
y = df_hepatite['Baselinehistological staging']

## Pipeline dos Dados

In [99]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

**Pipeline com dados númericos**

Com a normalização dos dados o modelo de predição almentou em um valor de 6% sua acurácia.

In [100]:
pipeline_numerico = Pipeline([
    ('std_scaler', StandardScaler())
])

In [101]:
X_num = X.drop(['Gender', 'Fever', 'Nausea/Vomting', 'Headache', 'Diarrhea', 'FGba', 'Jaundice', 'Ep'], axis=1)
X_num_tr = pipeline_numerico.fit_transform(X_num)

**Pipeline com dados categóricos**

In [102]:
X_num_std = pipeline_numerico.fit_transform(X_num)

In [103]:
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

In [104]:
try:
    from sklearn.compose import ColumnTransformer
except ImportError:
    from future_encoders import ColumnTransformer # Scikit-Learn < 0.20

In [105]:
# lista com os atributos categóricos
atributos_cat = ['Gender', 'Fever', 'Nausea/Vomting', 'Headache', 'Diarrhea', 'FGba', 'Jaundice', 'Ep']
# lista com os atributos númericos
atributos_num = list(X.drop(['Gender', 'Fever', 'Nausea/Vomting', 'Headache', 'Diarrhea', 'FGba', 'Jaundice', 'Ep'], axis=1))

In [106]:
pipeline_num_cat = ColumnTransformer([
    ('num', pipeline_numerico, atributos_num),
    ('cat', OneHotEncoder(), atributos_cat)
])

In [107]:
X_final = pipeline_num_cat.fit_transform(X)

# SMOTE

In [108]:
strategy = {1:1500, 2:1500, 3:1500, 4:1500}
oversample = SMOTE(sampling_strategy=strategy)
X_final, y = oversample.fit_resample(X_final, y)

## Funções para executar os modelos

In [109]:
#função que retorna um dicionário com os valores dos resultados
def model_results(model, X_train, y_train, X_test, y_test,results_dict_aux):
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # medindo e armazenando acurácia e f1-scoreno dicionário
    # accuracy = model.score(X_test, y_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    # AUC = roc_auc_score(y_test, model.predict_proba(X_test), average='weighted', multi_class='ovo')
    CM = confusion_matrix(y_test, y_pred)

    results_dict_aux['accuracy'].append(accuracy)
    results_dict_aux['f1'].append(f1)
    #results_dict_aux['auc'].append(AUC)
    results_dict_aux['cm'].append(CM)
    
    #print(f"f1: %.6f\n" %(f1))
    #print(f"Accuracy: %.6f\n" %(accuracy))
    #print(f"AUC: %.6f" %(AUC))
    #print(f"CM: \n{CM} \n")
    '''
    print("-----------------------CURVA ROC---------------------")
    visualizer = ROCAUC(model, encoder={1:"Class 1", 2:"Class 2", 3:"Class 3", 4:"Class 4"})

    visualizer.fit(X_train, y_train)        
    visualizer.score(X_test, y_test)        
    visualizer.show()                       
    #print("-----------------------------------------------------\n")'''
    
    return results_dict_aux

## RandomizedSearch

Para cada modelo é implementada uma função do RandomizedSearch. Para ser aplicada em cada um dos 10 conjuntos de treino do 10-fold.

In [110]:
def Random_search(model, param_grid, X, y):

    # defining parameter range
    grid = RandomizedSearchCV(model, param_grid, cv=3,random_state=199, scoring='accuracy', n_jobs=5)
    # fitting the model for grid search
    grid.fit(X, y)
    #utilizando melhores parâmetros calculados pelo gridsearch
    dic_best = grid.best_params_
    
    return dic_best

**KNN**

In [111]:
#dicionário com parêmetros para o gridsearch
knn_param_grid = {
    'n_neighbors': [impar for impar in range(1,32) if (impar%2)!=0],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidian','manhattan','chebyshev']
}

**Decision Tree**

In [112]:
#dicionário com parêmetros para o gridsearch
dt_param_grid = {
    'max_depth': [x for x in range(1,32)],
    'criterion': ['gini', 'entropy'],
}

**MLP**

In [113]:
# dicionário com parêmetros para o gridsearch
mlp_param_grid = {
    'hidden_layer_sizes': [(100,), (50, 15, 5), (100, 25, 10)],
    'activation': ['tanh', 'relu', 'logistic'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'learning_rate': ['constant', 'adaptive'],
    'learning_rate_init': [0.1, 0.01, 0.001],
}

**SVM**

In [114]:
#dicionário com parêmetros para o gridsearch
svm_param_grid = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['rbf','sigmoid', 'linear'], #['linear', 'poly', 'rbf', 'sigmoid']
    'gamma': [1, 0.1, 0.01, 0.001]
}

**Regressão Logística Multimodal**

In [115]:
mlr_param_grid = {
    'penalty': ['l2', 'none'],
    'solver': ['newton-cg', 'sag', 'lbfgs']
}

**Random Forest**

In [116]:
# profundidade máxima
max_depth = [int(x) for x in np.linspace(100, 300, num = 11)]
max_depth.append(None)
# grid
rf_param_grid = {
 'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 500, num = 10)],
 'max_depth': max_depth,
 # numero de features a serem consideradas em cada fold
 'max_features': ['auto', 'sqrt', 'log2']
}

**Gradiente Boosting**

In [117]:
gb_param_grid = {
    "loss":["deviance", "exponential"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
}

**Extreme Gradient Boosting**

In [118]:
xgb_param_grid = {
    "objective": ['multi:softmax'],
    "max_depth": range (2, 10, 1),
    "n_estimators": range(60, 220, 40),
    "learning_rate": [0.1, 0.01, 0.05],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

**Ada Boosting**

In [119]:
ab_param_grid = {
    'n_estimators':[int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)],
    'learning_rate':[.001,0.01, 0.1, 1, 2]
}

## Hold-out

Antes de realizar o 10-Fold os dados serão separados em treino e teste através do método **hold-out**. O data set de treino será utilizado para validar os modelos e escolher os melhores parâmetros com o **10-fold** para então comparar o modelos com o data set de teste.

In [120]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=199, stratify=y)

In [121]:
# converte para numpy array
y_train = np.array(y_train)
y_test = np.array(y_test)

## 10-Fold

Com esse método são criados 10 datasets de treino e 10 datasets de test com uma divisão de 90% para treino e 10% para teste em cada divisão.

O conjunto de treino sera divido mais uma vez em treino e validação (isso é feito dendo da função `GridSearchCV` para que então seja aplicado o GridSearch e assim obtenha-se os melhores parâmetros. Por fim, tendo os melhores parâmetros, utiliza-se o conjunto de teste para que se possa avaliar os resultados.

Esses resultados são obtidos de cada fold e então se tira a média deles para obter-se a avaliação final de cada modelo.

In [122]:
def calculate_mean_restults(results_dict_models):    
    # a cada interação calcula a média e o desvio padrão da 
    # acurácia, f1-score e matriz de confusão de cada modelo
    for model_key in results_dict_models.keys():
        accuracies = np.array(results_dict_models[model_key]['accuracy'])
        f1 = np.array(results_dict_models[model_key]['f1'])
        #auc = np.array(results_dict_models[model_key]['auc'])
        conf_matrix = np.array(results_dict_models[model_key]['cm'])

        print_mean_result(model_key, accuracies, f1, conf_matrix)

In [123]:
def print_best_results(best_dict_knn, best_dict_dt, best_dict_mlp, best_dict_svm, best_dict_mlr, best_dict_rf, best_dict_xgb, best_dict_gb , best_dict_ab):
    print("\n------- BEST PARAMETERS -------")
    print(f"KNN: {best_dict_knn}")
    print(f"DT: {best_dict_dt}")
    print(f"MLP: {best_dict_mlp}")
    print(f"SVM: {best_dict_svm}")
    print(f"MLR: {best_dict_mlr}")
    print(f"RF: {best_dict_rf}") 
    print(f"XGB: dic:{best_dict_xgb}") 
    print(f"GB: dic:{best_dict_gb}") 
    print(f"AB: dic:{best_dict_ab}")
    print("--------------------------------------------------------------------------")

In [124]:
def print_mean_result(model_key, accuracies, f1, conf_matrix):
    print(f"\t{model_key}")
    print("Acurácia média (desvio): %.6f +- (%.6f)" %(accuracies.mean(), accuracies.std()))
    print("F1-score média (desvio): %.6f +- (%.6f)" %(f1.mean(), f1.std()))
    #print("AUC média (desvio): %.6f +- (%.6f)\n" %(auc.mean(), auc.std()))
    print(f"Matriz de Confusão:  \n{sum(conf_matrix)*0.1}")

In [125]:
#função que roda os modelos em cada uma das divisões do 10-fold
#e imprime a média e o desvio padrão dos resultados

def evaluate_model_with_kfold(kf):
    results_dict_models = {}
    # listas e dicionarios para salvar as métricas dos resultados de todas as interacoes
    # a key 'best'salva a melhor acurácia
    
    results_dict_KNN = {
        'accuracy': [],
        'f1': [],
        #'auc': [],
        'cm': [],
        'best': 0.0
    }
    
    results_dict_PERC = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_DT = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_MLP = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_SVM = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_GNB = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_MLR = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_RF = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_XGB = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_GB = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }
    results_dict_AB = {
        'accuracy': [], 'f1': [],  'cm': [], 'best': 0.0
    }

    
    # váriável para salvar os melhores parâmetros
    best_dict_knn, best_dict_dt, best_dict_mlp, best_dict_svm, best_dict_mlr, best_dict_rf, best_dict_xgb, best_dict_gb , best_dict_ab = {}, {}, {}, {}, {}, {}, {}, {}, {}
    
    fold = 0
    
    # laço que roda todos os modelos em cada 1-fold
    for train, test in kf.split(X_train, y_train):
        
        # usa a lista retornada pelo .split para selicionar as intâncias de cada fold
        X_train_kf = X_train[train,:] 
        y_train_kf = y_train[train]
        X_test_kf = X_train[test,:]
        y_test_kf = y_train[test]
        
        # para acompanhar a execução 
        fold += 1
        print(f"\n{fold}º fold")
        
        # Método de seleção de features não supervisionado
        # Limiar de variância - retira-se as features com variância menor que 85%
        filter_variance = VarianceThreshold(0.85)
        filter_variance.fit(X_train_kf)
        X_train_kf = filter_variance.transform(X_train_kf)
        X_test_kf = filter_variance.transform(X_test_kf)
        print("Features selecionadas: %d" %(X_train_kf.shape[1]))
        
        #kNN
        print("-KNN")
        dict_knn = Random_search(knn(), knn_param_grid, X_train_kf, y_train_kf)
        model = knn(**dict_knn)
        results_dict_KNN = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_KNN)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_KNN['accuracy'][-1] > results_dict_KNN['best']): 
            results_dict_KNN['best'] = results_dict_KNN['accuracy'][-1]
            best_dict_knn = dict_knn 
            
        #PerC
        print("-PerC")
        model = PerC()
        results_dict_PERC = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_PERC)
            
        #DT
        print("-DT")
        dict_dt = Random_search(DecisionTreeClassifier(), dt_param_grid, X_train_kf, y_train_kf)
        model = DecisionTreeClassifier(**dict_dt, random_state=199)
        results_dict_DT = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_DT)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_DT['accuracy'][-1] > results_dict_DT['best']): 
            results_dict_DT['best'] = results_dict_DT['accuracy'][-1]
            best_dict_dt = dict_dt
        
        
        #MLP 
        print("-MLP")
        dict_mlp = Random_search(MLPClassifier(),mlp_param_grid, X_train_kf,y_train_kf)
        model = MLPClassifier(**dict_mlp, max_iter=2000, tol=0.000001, random_state=199)
        results_dict_MLP = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_MLP)
        #verifica se os parêmetros do fold atual são os melhores comparando a acurácia
        if (results_dict_MLP['accuracy'][-1] > results_dict_MLP['best']): 
            results_dict_MLP['best'] = results_dict_MLP['accuracy'][-1]
            best_dict_mlp = dict_mlp
        
        #GNB 
        print("-GNB")
        model = GaussianNB()
        results_dict_models['GNB'] = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_GNB)
        
        #SVM
        print("-SVM")
        dict_svm = Random_search(SVC(), svm_param_grid, X_train_kf, y_train_kf)
        model = SVC(**dict_svm, probability=True, random_state=199)
        results_dict_SVM = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_SVM)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_SVM['accuracy'][-1] > results_dict_SVM['best']): 
            results_dict_SVM['best'] = results_dict_SVM['accuracy'][-1]
            best_dict_svm = dict_svm           
        
        # Regressão Logística Multimodal
        print("-MLR")
        dict_mlr = Random_search(LogisticRegression(), mlr_param_grid, X_train_kf, y_train_kf)
        model = LogisticRegression(**dict_mlr, multi_class='multinomial', random_state=199)
        results_dict_MLR = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_MLR)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_MLR['accuracy'][-1] > results_dict_MLR['best']): 
            results_dict_MLR['best'] = results_dict_MLR['accuracy'][-1]
            best_dict_mlr = dict_mlr 
        
        # Random Forest
        print("-RF")
        dict_rf = Random_search(RandomForestClassifier(), rf_param_grid, X_train_kf, y_train_kf)
        model = RandomForestClassifier(**dict_rf, random_state=199)
        results_dict_RF = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_RF)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_RF['accuracy'][-1] > results_dict_RF['best']): 
            results_dict_RF['best'] = results_dict_RF['accuracy'][-1]
            best_dict_rf = dict_rf 
      
        # Ada Boosting
        print("-AB")
        dict_ab = Random_search(AdaBoostClassifier(), ab_param_grid, X_train_kf, y_train_kf)
        model = AdaBoostClassifier(**dict_ab, random_state=199)
        results_dict_AB = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_AB)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_AB['accuracy'][-1] > results_dict_AB['best']): 
            results_dict_AB['best'] = results_dict_AB['accuracy'][-1]
            best_dict_ab = dict_ab
      
        # EXtrem Gradiente Boosting
        print("-XGB")
        dict_xgb = Random_search(XGBClassifier(), xgb_param_grid, X_train_kf, y_train_kf)
        model = XGBClassifier(**dict_xgb, random_state=199)
        results_dict_XGB = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_XGB)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_XGB['accuracy'][-1] > results_dict_XGB['best']): 
            results_dict_XGB['best'] = results_dict_XGB['accuracy'][-1]
            best_dict_xgb = dict_xgb
      
        # GB
        print("-GB")
        dict_gb = Random_search(GradientBoostingClassifier(), gb_param_grid, X_train_kf, y_train_kf)
        model = GradientBoostingClassifier(**dict_gb, random_state=199)
        results_dict_GB = model_results(model, X_train_kf, y_train_kf, X_test_kf, y_test_kf, results_dict_GB)
        #verifica se os parêmetros do fold atual são os melhores
        if (results_dict_GB['accuracy'][-1] > results_dict_GB['best']): 
            results_dict_GB['best'] = results_dict_GB['accuracy'][-1]
            best_dict_gb = dict_gb        
        
    results_dict_models['KNN'] = results_dict_KNN
    results_dict_models['DT'] = results_dict_DT
    results_dict_models['MLP'] = results_dict_MLP
    results_dict_models['GNB'] = results_dict_GNB
    results_dict_models['SVM'] = results_dict_SVM
    results_dict_models['MLR'] = results_dict_MLR
    results_dict_models['RF'] = results_dict_RF
    results_dict_models['XGB'] = results_dict_XGB
    results_dict_models['GB'] = results_dict_GB
    results_dict_models['AB'] = results_dict_AB
    
    # calcula a média dos resultados e imprime cada métrica
    calculate_mean_restults(results_dict_models)

    # imprime os melhores parâmetros
    print_best_results(best_dict_knn, best_dict_dt, best_dict_mlp, best_dict_svm, best_dict_mlr, best_dict_rf, best_dict_xgb, best_dict_ab, best_dict_gb)
    
    # salva os melhores parâmetros em um dicionário que é o retorno da função
    parameters_dict = {
        'knn': best_dict_knn,
        'dt': best_dict_dt,
        'mlp': best_dict_mlp,
        'svm': best_dict_svm,
        'mlr': best_dict_mlr,
        'rf': best_dict_rf,
        'xgb': best_dict_xgb,
        'gb': best_dict_gb,
        'ab': best_dict_ab
    }
    
    return parameters_dict

In [126]:
import warnings

In [127]:
%%time
#ignorando warnings
warnings.filterwarnings('ignore')

params_dict = {}

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=199)
params_dict = evaluate_model_with_kfold(skf)


1º fold
Features selecionadas: 4
-KNN
-PerC
-DT
-MLP
-GNB
-SVM
-MLR
-RF
-AB
-XGB
[23:00:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:00:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
-GB

2º fold
Features selecionadas: 4
-KNN
-PerC
-DT
-MLP
-GNB
-SVM
-MLR
-RF
-AB
-XGB
[23:02:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was change

## Teste

Aqui os modelos são execultados no dataset de teste com seus melhores parâmetros.

In [149]:
def model_test_results(model, X_train, y_train, X_test, y_test):
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #medindo e armazenando acurácia e f1-score no dicionário
    
    #accuracy = model.score(X_test, y_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    #AUC = roc_auc_score(y_test, model.predict_proba(X_test), average='weighted', multi_class='ovo')
    CM = confusion_matrix(y_test, y_pred)
    
    print(f"Accuracy: %.6f" %(accuracy))
    print(f"f1: %.6f" %(f1))
    #print(f"AUC: %.6f" %(AUC))
    print(f"CM: \n{CM} \n")

**KNN**

In [150]:
best_param_dict = params_dict['knn']

In [130]:
model = knn(**best_param_dict)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.978889
f1: 0.978899
CM: 
[[439   0   3   8]
 [  3 438   4   5]
 [  3   3 441   3]
 [  1   4   1 444]] 



**PerC**

In [131]:
model = PerC()
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.576667
f1: 0.577598
CM: 
[[262  55  85  48]
 [ 71 238  91  50]
 [ 86  46 279  39]
 [ 67  41  83 259]] 



**DT**

In [132]:
best_param_dict = params_dict['dt']

In [133]:
model = DecisionTreeClassifier(**best_param_dict, random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.687778
f1: 0.687718
CM: 
[[313  55  47  35]
 [ 51 301  58  40]
 [ 48  51 298  53]
 [ 39  38  47 326]] 



**MLP**

In [134]:
best_param_dict = params_dict['mlp']

In [135]:
model = MLPClassifier(
            **best_param_dict, 
            max_iter=2000, 
            tol=0.000001,
            random_state=199
        )
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.908333
f1: 0.908346
CM: 
[[414  12   9  15]
 [ 14 414   9  13]
 [ 12  14 404  20]
 [ 18  17  12 403]] 



**GNB**

In [136]:
model = GaussianNB()
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.373333
f1: 0.373617
CM: 
[[167 116  88  79]
 [ 89 178  96  87]
 [118 108 153  71]
 [104 107  65 174]] 



**SVM**

In [137]:
best_param_dict = params_dict['svm']

In [138]:
model = SVC(**best_param_dict, probability=True, random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.925000
f1: 0.927691
CM: 
[[450   0   0   0]
 [ 64 386   0   0]
 [ 42   0 408   0]
 [ 29   0   0 421]] 



**MLR**

In [139]:
best_param_dict = params_dict['mlr']

In [140]:
model = LogisticRegression(**best_param_dict, multi_class='multinomial', random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.360000
f1: 0.357781
CM: 
[[133 122  94 101]
 [ 69 184 104  93]
 [ 89 120 137 104]
 [ 79 116  61 194]] 



**RF**

In [141]:
best_param_dict = params_dict['rf']

In [142]:
model = RandomForestClassifier(**best_param_dict, random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.952222
f1: 0.952269
CM: 
[[430   3   6  11]
 [  5 423   7  15]
 [  2  14 429   5]
 [  4   7   7 432]] 



**XGB**

In [143]:
best_param_dict = params_dict['xgb']

In [144]:
model = XGBClassifier(**best_param_dict, random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

[23:17:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 0.901667
f1: 0.901667
CM: 
[[410  17  16   7]
 [ 12 404  17  17]
 [ 11  16 408  15]
 [ 19  16  14 401]] 



**Gradient Boost**

In [145]:
best_param_dict = params_dict['gb']

In [146]:
model = GradientBoostingClassifier(**best_param_dict, random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.324444
f1: 0.311514
CM: 
[[172  84 163  31]
 [116 128 167  39]
 [135  60 217  38]
 [111 112 160  67]] 



**Ada Boost**

In [147]:
best_param_dict = params_dict['ab']

In [148]:
model = AdaBoostClassifier(**best_param_dict, random_state=199)
model_test_results(model, X_train, y_train, X_test, y_test)

Accuracy: 0.380556
f1: 0.379958
CM: 
[[152 106 102  90]
 [ 77 185  90  98]
 [107  83 159 101]
 [ 93  88  80 189]] 

